In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex客户端库：用原始字节输入进行在线预测的自定义训练图像分类模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_image_classification_online_raw.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_image_classification_online_raw.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex客户端库来训练和部署用于在线预测的自定义原始字节模型。其目的是演示当输入到服务函数是原始字节（而不是图像、文本或表格格式）时，该服务函数是如何构建的。为了演示目的，底层模型是一个图像分类模型，并且在服务函数中重建了图像。

数据集

本教程使用的数据集是[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/cifar10)中的[CIFAR10数据集](https://www.tensorflow.org/datasets/catalog/overview)。您将使用的数据集版本已内置于TensorFlow中。训练后的模型可以预测图像属于十个类别中的哪一类：飞机、汽车、鸟、猫、鹿、狗、青蛙、马、船、卡车。

### 目标

在本教程中，您将学习如何使用Vertex客户端库在Docker容器中从Python脚本创建自定义模型，然后通过发送原始字节数据对部署的模型进行预测。您也可以使用`gcloud`命令行工具或通过Google Cloud控制台在线创建和部署模型。


所执行的步骤包括：

- 创建用于训练模型的Vertex自定义作业。
- 使用自定义容器训练TensorFlow模型。
- 检索和加载模型工件。
- 查看模型评估。
- 构建用于预处理原始字节的服务功能。
- 将模型上传为Vertex `Model`资源。
- 将`Model`资源部署到服务`Endpoint`资源。
- 进行预测。
- 取消部署`Model`资源。

### 价格

本教程使用Google Cloud（GCP）的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)，根据您的预期使用情况生成费用估算。

安装

安装最新版的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的 *google-cloud-storage* 库的 GA 版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动内核

当您安装了Vertex客户端库和Google *cloud-storage*后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

开始之前

GPU运行时

*确保在具有此选项的GPU运行时中运行此笔记本。在Colab中，选择* **Runtime > Change Runtime Type > GPU**

设置您的Google Cloud项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得$300免费信用用于您的计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用Vertex APIs和Compute Engine APIs。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已经安装在Google Cloud Notebook中。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，确保
Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter将以 `!` 为前缀的行视为 shell 命令，并且将以 `$` 为前缀的Python变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改 `REGION` 变量，该变量用于操作本笔记本其余部分。以下是 Vertex 支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您不能使用多区域存储桶进行 Vertex 的培训。并非所有区域都支持所有 Vertex 服务。有关每个区域的最新支持信息，请参阅[Vertex 位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行实时教程会话，您可能正在使用共享的测试帐户或项目。为了避免用户在创建的资源中发生名称冲突，您为每个实例会话创建一个时间戳，并附加到在此教程中将要创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云账户

**如果您正在使用谷歌云笔记本**，您的环境已经通过验证。跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，按照提示进行验证您的账户的oAuth。

**否则**，请按照以下步骤操作：

在 Cloud 控制台中，转到 [创建服务账户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey) 页面。

**点击创建服务帐户**。

在**服务帐户名称**字段中，输入一个名称，然后点击**创建**。

在**将此服务帐户访问授予权给项目**部分，点击角色下拉列表。在筛选框中输入 "Vertex"，然后选择**Vertex 管理员**。在筛选框中输入 "Storage Object Admin"，然后选择**存储对象管理员**。

点击创建。一个包含您密钥的 JSON 文件将下载到您本地环境。

在下面的单元格中将路径输入到您的服务账户密钥作为 GOOGLE_APPLICATION_CREDENTIALS 变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论你在哪个笔记本环境中，都需要按照以下步骤进行操作。**

当您使用 Vertex 客户端库提交自定义训练作业时，您需要上传一个包含训练代码的 Python 包到一个云存储桶中。Vertex 将会从这个包中运行代码。在本教程中，Vertex 也会将训练作业得到的训练模型保存到同一个存储桶中。然后您可以基于这个输出创建一个 `Endpoint` 资源，以便进行在线预测。

请设置您的云存储桶的名称。存储桶的名称必须在所有 Google Cloud 项目中全局唯一，包括您组织外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶尚不存在时才能运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查其内容来验证对云存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

#### 导入顶点客户端库

将顶点客户端库导入到我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

#### 顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：顶点API服务的数据集、模型、作业、流水线和端点服务端点。
- `PARENT`：顶点数据集、模型、作业、流水线和端点资源的根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### 硬件加速器

设置硬件加速器（例如GPU）用于训练和预测。

设置变量`TRAIN_GPU / TRAIN_NGPU`和`DEPLOY_GPU / DEPLOY_NGPU`以使用支持GPU的容器映像和分配给虚拟机（VM）实例的GPU数量。 例如，要使用一个设置为每个VM分配了4个Nvidia Telsa K80 GPU的GPU容器映像，您可以指定：

（aip.AcceleratorType.NVIDIA_TESLA_K80，4）

对于GPU，可用的加速器包括：
   - aip.AcceleratorType.NVIDIA_TESLA_K80
   - aip.AcceleratorType.NVIDIA_TESLA_P100
   - aip.AcceleratorType.NVIDIA_TESLA_P4
   - aip.AcceleratorType.NVIDIA_TESLA_T4
   - aip.AcceleratorType.NVIDIA_TESLA_V100

否则，指定`(None, None)`以使用一个在CPU上运行的容器映像。

*注意*：为GPU支持的TF 2.3之前的版本将无法加载本教程中的自定义模型。这是一个已知问题，在TF 2.3中已得到解决——由于在服务函数中生成的静态图操作引起的。如果在您自己的自定义模型上遇到此问题，请使用支持GPU的TF 2.3的容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

容器（Docker）镜像

接下来，我们将设置用于训练和预测的Docker容器镜像

- TensorFlow 1.15
  - `gcr.io/cloud-aiplatform/training/tf-cpu.1-15:latest`
  - `gcr.io/cloud-aiplatform/training/tf-gpu.1-15:latest`
- TensorFlow 2.1
  - `gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest`
  - `gcr.io/cloud-aiplatform/training/tf-gpu.2-1:latest`
- TensorFlow 2.2
  - `gcr.io/cloud-aiplatform/training/tf-cpu.2-2:latest`
  - `gcr.io/cloud-aiplatform/training/tf-gpu.2-2:latest`
- TensorFlow 2.3
  - `gcr.io/cloud-aiplatform/training/tf-cpu.2-3:latest`
  - `gcr.io/cloud-aiplatform/training/tf-gpu.2-3:latest`
- TensorFlow 2.4
  - `gcr.io/cloud-aiplatform/training/tf-cpu.2-4:latest`
  - `gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest`
- XGBoost
  - `gcr.io/cloud-aiplatform/training/xgboost-cpu.1-1`
- Scikit-learn
  - `gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23:latest`
- Pytorch
  - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-4:latest`
  - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-5:latest`
  - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-6:latest`
  - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-7:latest`

有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)。

- TensorFlow 1.15
  - `gcr.io/cloud-aiplatform/prediction/tf-cpu.1-15:latest`
  - `gcr.io/cloud-aiplatform/prediction/tf-gpu.1-15:latest`
- TensorFlow 2.1
  - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-1:latest`
  - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-1:latest`
- TensorFlow 2.2
  - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest`
  - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-2:latest`
- TensorFlow 2.3
  - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-3:latest`
  - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-3:latest`
- XGBoost
  - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-2:latest`
  - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-1:latest`
  - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-90:latest`
  - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-82:latest`
- Scikit-learn
  - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-23:latest`
  - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-22:latest`
  - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-20:latest`

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量 `TRAIN_COMPUTE` 和 `DEPLOY_COMPUTE` 来配置您将用于训练和预测的虚拟机的计算资源。
 - 机器类型
     - `n1-standard`：每个vCPU 3.75GB的内存。
     - `n1-highmem`：每个vCPU 6.5GB的内存
     - `n1-highcpu`：每个vCPU 0.9GB的内存
 - vCPUs：\[2, 4, 8, 16, 32, 64, 96\]的数量

*注意：以下不支持用于训练:*

 - `standard`：2个vCPUs
 - `highcpu`：2, 4和8个vCPUs

*注意：您也可以使用n2和e2机器类型用于训练和部署，但它们不支持GPU。*

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在您已经准备好开始创建自己的定制模型并针对CIFAR10进行训练。

## 设置客户端

Vertex客户端库以客户端/服务器模型运行。在您的一侧（Python脚本），您将创建一个客户端，发送请求并接收来自Vertex服务器的响应。

在本教程中，您将为工作流程中的不同步骤使用不同的客户端。因此，请提前设置它们。

- 为`模型`资源设置Model服务。
- 部署设置Endpoint服务。
- 批处理作业和自定义训练设置Job服务。
- 为提供服务设置Prediction服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["job"] = create_job_client()
clients["model"] = create_model_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

训练模型

有两种方法可以使用容器镜像来训练自定义模型：

- **使用 Google Cloud 预构建的容器**。如果使用预构建的容器，您还需要指定一个要安装到容器镜像中的 Python 包。这个 Python 包包含了您用于训练自定义模型的代码。

- **使用您自己的自定义容器镜像**。如果使用您自己的容器，容器需要包含您用于训练自定义模型的代码。

准备您的定制工作规范

现在您的客户已经准备好了，您的第一步是为您的定制培训工作创建一个工作规范。工作规范将包括以下内容：

- `worker_pool_spec`：您将用于培训的机器类型和数量的规范（单个或分布式）
- `python_package_spec`：要安装在预构建容器中的Python软件包规范

### 准备您的机器规格

现在为您定制的训练工作定义机器规格。这会告诉Vertex要为训练提供什么类型的机器实例。
  - `machine_type`：要提供的GCP实例的类型--例如，n1-standard-8。
  - `accelerator_type`：如果有的话，硬件加速器的类型。在本教程中，如果您之前设置了变量`TRAIN_GPU != None`，则表示您正在使用GPU；否则您将使用CPU。
  - `accelerator_count`：加速器的数量。

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU,
    }
else:
    machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

### 准备您的磁盘规格

（可选）现在定义您自定义训练作业的磁盘规格。这将告诉 Vertex 在每个机器实例中为训练分配什么类型和大小的磁盘。

- `boot_disk_type`：可选SSD或Standard。SSD更快，Standard更便宜。默认为SSD。
- `boot_disk_size_gb`：磁盘的大小（以GB为单位）。

In [ ]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 200  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

###定义工作池规范

接下来，您需要为自定义训练作业定义工作池规范。工作池规范将包括以下内容：

- `replica_count`：要启动的此机器类型的实例数。
- `machine_spec`：硬件规格。
- `disk_spec`：（可选）磁盘存储规格。

- `python_package`：要安装在VM实例上的Python训练包，以及要调用的Python模块，以及Python模块的命令行参数。

现在让我们深入了解Python包规范：

- `executor_image_spec`：这是为您的自定义训练作业配置的Docker镜像。

- `package_uris`：这是要安装在已配置实例上的Python训练包的位置（URI）列表。位置需要位于Cloud Storage存储桶中。 这些可以是单个Python文件，也可以是整个包的zip（存档）。 在后一种情况下，作业服务将把内容解压缩（取消存档）到Docker镜像中。

- `python_module`：用于运行自定义训练作业的Python模块（脚本）。 在此示例中，您将调用 `trainer.task.py` -- 请注意，不需要附加 `.py` 后缀。

- `args`：要传递给相应Python模块的命令行参数。 在此示例中，您将设置：
  - `"--model-dir=" + MODEL_DIR`：保存模型工件的Cloud Storage位置。 有两种方法告诉训练脚本在哪里保存模型工件：
      - 直接：您将Cloud Storage位置作为命令行参数传递给训练脚本（设置变量 `DIRECT = True`），
      - 间接：服务将Cloud Storage位置作为环境变量 `AIP_MODEL_DIR` 传递给训练脚本（设置变量 `DIRECT = False`）。 在这种情况下，您需要在作业规范中告诉服务模型工件的位置。
  - `"--epochs=" + EPOCHS`：训练的时代数。
  - `"--steps=" + STEPS`：每时代的步数（批处理数）。
  - `"--distribute=" + TRAIN_STRATEGY"`：用于单个或分布式训练的训练分发策略。
     - `"single"`：单个设备。
     - `"mirror"`：单个计算实例上的所有 GPU 设备。
     - `"multi"`：所有计算实例上的所有 GPU 设备。

In [ ]:
JOB_NAME = "custom_job_" + TIMESTAMP
MODEL_DIR = "{}/{}".format(BUCKET_NAME, JOB_NAME)

if not TRAIN_NGPU or TRAIN_NGPU < 2:
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "mirror"

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "disk_spec": disk_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_NAME + "/trainer_cifar10.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

### 组装工作规范

现在组装定制工作规范的完整描述：

- `display_name`: 您指定给这个定制工作的可读名称。
- `job_spec`: 定制工作的规范。
    - `worker_pool_specs`: 用于机器虚拟机实例的规范。
    - `base_output_directory`: 这告诉服务在哪里保存模型工件的云存储位置（当变量 `DIRECT = False` 时）。然后服务将将位置传递给训练脚本作为环境变量 `AIP_MODEL_DIR`，路径将以以下形式显示：

                <output_uri_prefix>/model

In [ ]:
if DIRECT:
    job_spec = {"worker_pool_specs": worker_pool_spec}
else:
    job_spec = {
        "worker_pool_specs": worker_pool_spec,
        "base_output_directory": {"output_uri_prefix": MODEL_DIR},
    }

custom_job = {"display_name": JOB_NAME, "job_spec": job_spec}

### 检查培训包

#### 包布局

在开始培训之前，您将查看一个Python包是如何为定制培训任务组装的。解压后，包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

`setup.cfg` 和 `setup.py` 文件是将包安装到Docker镜像操作环境中的指令。

文件 `trainer/task.py` 是执行定制培训任务的Python脚本。*注意*，当我们在工作池规范中引用它时，我们将目录斜杠替换为点(`trainer.task`)，并删除文件后缀(`.py`)。

#### 包装配

在下面的单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: CIFAR10 image classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py内容

在下一个单元格中，您将编写训练脚本task.py的内容。我们不会详细讨论，只是让您浏览一下。总结如下：

- 从命令行获取保存模型文件的目录（`--model_dir`），如果没有指定，则从环境变量`AIP_MODEL_DIR`中获取。
- 从TF Datasets（tfds）加载CIFAR10数据集。
- 使用TF.Keras模型API构建模型。
- 编译模型（`compile()`）。
- 根据参数`args.distribute`设置训练分布策略。
- 根据参数`args.epochs`和`args.steps`训练模型（`fit()`）。
- 将训练好的模型（`save(args.model_dir)`）保存到指定的模型目录中。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for CIFAR-10

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv("AIP_MODEL_DIR"), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.01, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print('DEVICES', device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64


def make_datasets_unbatched():

  # Scaling CIFAR10 data from (0, 255] to (0., 1.]
  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label


  datasets, info = tfds.load(name='cifar10',
                            with_info=True,
                            as_supervised=True)
  return datasets['train'].map(scale).cache().shuffle(BUFFER_SIZE).repeat()


# Build the Keras model
def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      optimizer=tf.keras.optimizers.SGD(learning_rate=args.lr),
      metrics=['accuracy'])
  return model


# Train the model
NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS
train_dataset = make_datasets_unbatched().batch(GLOBAL_BATCH_SIZE)

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_cnn_model()

model.fit(x=train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
model.save(args.model_dir)

将培训脚本存储在您的云存储桶中

接下来，您将培训文件夹打包成压缩的tar包，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_cifar10.tar.gz

### 训练模型

现在开始在Vertex上训练您的自定义训练作业。使用以下帮助函数`create_custom_job`，该函数接受以下参数：

- `custom_job`：自定义作业的规范。

帮助函数调用作业客户端服务的`create_custom_job`方法，参数如下：

- `parent`：指向`Dataset`，`Model`和`Endpoint`资源的Vertex位置路径。
- `custom_job`：自定义作业的规范。

您将在`response`对象中显示返回的一些字段，其中最感兴趣的两个是：

`response.name`：分配给此自定义训练作业的Vertex完全限定标识符。您将保存此标识符以在后续步骤中使用。

`response.state`：自定义训练作业的当前状态。

In [ ]:
def create_custom_job(custom_job):
    response = clients["job"].create_custom_job(parent=PARENT, custom_job=custom_job)
    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = create_custom_job(custom_job)

现在获取您创建的自定义作业的唯一标识符。

In [ ]:
# The full unique ID for the custom job
job_id = response.name
# The short numeric ID for the custom job
job_short_id = job_id.split("/")[-1]

print(job_id)

### 获取自定义作业的信息

接下来，使用这个辅助函数 `get_custom_job`，它接受以下参数：

- `name`: 自定义作业的 Vertex 完全限定标识符。

辅助函数调用作业客户端服务的 `get_custom_job` 方法，传入以下参数：

- `name`: 自定义作业的 Vertex 完全限定标识符。

如果您记得，在调用 `create_custom_job` 方法时，在 `response.name` 字段中获取了自定义作业的 Vertex 完全限定标识符，并将该标识符保存在变量 `job_id` 中。

In [ ]:
def get_custom_job(name, silent=False):
    response = clients["job"].get_custom_job(name=name)
    if silent:
        return response

    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = get_custom_job(job_id)

部署

训练以上模型可能需要超过20分钟的时间。

一旦您的模型训练完成，您可以通过将`end_time`减去`start_time`来计算训练模型所需的实际时间。对于您的模型，我们需要知道保存模型的位置，Python脚本将保存在您的本地云存储桶中的`MODEL_DIR + '/saved_model.pb'`。

In [ ]:
while True:
    response = get_custom_job(job_id, True)
    if response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_path_to_deploy = None
        if response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        if not DIRECT:
            MODEL_DIR = MODEL_DIR + "/model"
        model_path_to_deploy = MODEL_DIR
        print("Training Time:", response.update_time - response.create_time)
        break
    time.sleep(60)

print("model_to_deploy:", model_path_to_deploy)

加载保存的模型

您的模型以TensorFlow SavedModel格式存储在Cloud Storage存储桶中。现在从Cloud Storage存储桶加载它，然后您可以做一些事情，比如评估模型和进行预测。

要加载，您可以使用TF.Keras `model.load_model()`方法，将保存模型的Cloud Storage路径传递给它，该路径由`MODEL_DIR`指定。

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在找出模型的表现如何。

### 加载评估数据

你将从 `tf.keras.datasets` 加载 CIFAR10 测试（留出）数据，使用 `load_data()` 方法。这将返回数据集作为两个元素的元组。第一个元素是训练数据，第二个是测试数据。每个元素也是两个元素的元组：图像数据和相应的标签。

你不需要训练数据，这就是为什么我们加载它为 `(_,_)`。

在运行数据进行评估之前，你需要预处理数据：

x_test:
1. 将像素数据进行归一化（重新缩放），通过将每个像素除以 255。这将用 0 到 1 之间的 32 位浮点数替换每个单个字节整数像素。

y_test:
2. 标签目前是标量（稀疏）的。如果你回顾 `trainer/task.py` 脚本中的 `compile()` 步骤，你会发现它是为稀疏标签编译的。因此我们无需做任何其他操作。

In [ ]:
import numpy as np
from tensorflow.keras.datasets import cifar10

(_, _), (x_test, y_test) = cifar10.load_data()
x_test = (x_test / 255.0).astype(np.float32)

print(x_test.shape, y_test.shape)

### 进行模型评估

现在评估定制作业中模型的表现如何。

In [ ]:
model.evaluate(x_test, y_test)

## 上传模型进行服务

接下来，您将把 TF.Keras 模型从自定义作业上传到 Vertex `Model` 服务，该服务将为您的自定义模型创建一个 Vertex `Model` 资源。在上传过程中，您需要定义一个服务函数，将数据转换为您的模型所期望的格式。如果您向 Vertex 发送编码数据，您的服务函数将确保在数据传递到模型之前，在模型服务器上对数据进行解码。

### 服务函数如何工作

当您向在线预测服务器发送请求时，请求将被 HTTP 服务器接收。HTTP 服务器从 HTTP 请求内容体中提取预测请求。提取的预测请求将被转发到服务函数。对于 Google 预构建的预测容器，请求内容将作为 `tf.string` 传递给服务函数。

服务函数由两部分组成：

- `预处理函数`：
  - 将输入（`tf.string`）转换为基础模型（动态图）的输入形状和数据类型。
  - 执行与训练基础模型期间相同的数据预处理 -- 例如，归一化，缩放等。
- `后处理函数`：
  - 将模型输出转换为接收应用程序所期望的格式 -- 例如，压缩输出。
  - 打包输出以供接收应用程序使用 -- 例如，添加标题，生成 JSON 对象等。

预处理和后处理函数都会被转换为与模型融合的静态图。基础模型的输出会传递给后处理函数。后处理函数将转换/打包输出传递回 HTTP 服务器。HTTP 服务器将输出作为 HTTP 响应内容返回。

在构建 TF.Keras 模型的服务函数时需要考虑的一个问题是它们作为静态图运行。这意味着您不能使用需要动态图的 TF 图操作。如果使用了这样的操作，您将在服务函数的编译过程中收到错误提示，指出您正在使用不受支持的 EagerTensor。

### 原始字节的服务功能

为了向预测服务传递原始字节，您需要将字节编码为base64 - 这样可以使内容在通过网络传输二进制数据时免受修改。由于部署的模型期望输入数据为原始字节，您需要确保base64编码的数据在传递给部署的模型作为输入之前被转换回原始字节。

为了解决这个问题，定义一个服务功能（`serving_fn`）并将其附加到模型作为预处理步骤。添加一个`@tf.function`装饰器，这样服务功能将被融合到底层模型中（而不是在CPU上游）。

当您发送预测或解释请求时，请求的内容会被base64解码成一个TensorFlow字符串，然后传递给服务功能（`serving_fn`）。服务功能对tf.string进行预处理，将其预处理为原始的numpy字节（`preprocess_fn`）以匹配模型的输入要求：

- `tf.io.decode_raw`：将base64编码的tf.string解码为解码后的字节张量。
- `tf.reshape`：将字节张量重塑为（32，32，3）- 即CIFAR10。
- `tf.cast`：将整数像素值转换为浮点数。
- `重塑/255.0`：将像素数据重新缩放（归一化）到0和1之间。

在这一点上，数据可以传递给模型（`m_call`）。

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_raw(bytes_input, tf.uint8)
    resized = tf.reshape(decoded, shape=(32, 32, 3))
    recast = tf.cast(resized, tf.float32)
    rescale = tf.cast(recast / 255.0, tf.float32)
    return rescale


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


m_call = tf.function(model.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 32, 32, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


tf.saved_model.save(
    model,
    model_path_to_deploy,
    signatures={
        "serving_default": serving_fn,
    },
)

获取服务函数签名

您可以通过重新加载模型到内存，并查询每个层对应的签名来获取模型的输入和输出层的签名。

对于您的目的，您需要获取服务函数的签名。为什么呢？因为当我们将数据作为 HTTP 请求数据包发送进行预测时，图像数据是 base64 编码的，而我们的 TF.Keras 模型接受 numpy 输入。您的服务函数将负责将 base64 转换为 numpy 数组。

在进行预测请求时，您需要将请求路由到服务函数而不是模型，因此您需要了解服务函数的输入层名称--您之后在进行预测请求时会用到。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

### 上传模型

使用这个辅助函数`upload_model`来上传您的模型，存储在SavedModel格式中，到`Model`服务中，该服务将为您的模型实例化一个Vertex`Model`资源实例。一旦您完成了这一步，您可以像使用任何其他Vertex`Model`资源实例一样使用`Model`资源实例，例如部署到用于提供预测的`Endpoint`资源。

辅助函数接受以下参数：

- `display_name`：用于`Endpoint`服务的人类可读名称。
- `image_uri`：模型部署的容器镜像。
- `model_uri`：SavedModel艺术品的Cloud Storage路径。对于本教程，这是`trainer/task.py`保存了模型艺术品的Cloud Storage位置，我们在变量`MODEL_DIR`中指定了它。

辅助函数调用了`Model`客户端服务的方法`upload_model`，该方法接受以下参数：

- `parent`：用于`Dataset`、`Model`和`Endpoint`资源的Vertex位置根路径。
- `model`：Vertex`Model`资源实例的规范。

现在让我们更深入地了解Vertex模型规范`model`。这是一个包含以下字段的字典对象：

- `display_name`：用于`Model`资源的人类可读名称。
- `metadata_schema_uri`：由于您的模型是在没有Vertex`Dataset`资源的情况下构建的，您将保留为空（`''`）。
- `artificat_uri`：以SavedModel格式存储模型的Cloud Storage路径。
- `container_spec`：这是将安装在`Endpoint`资源上的Docker容器的规范，从中`Model`资源将提供预测。使用您之前设置的变量`DEPLOY_GPU != None`来使用GPU；否则只分配CPU。

将模型上传到Vertex Model资源会返回一个长时间运行的操作，因为这可能需要一些时间。您调用response.result()，这是一个同步调用，当Vertex Model资源准备就绪时将返回。

辅助函数会返回相应Vertex Model实例的完全合格标识符为upload_model_response.model。您将保存这个标识符用于后续步骤中的变量model_to_deploy_id。

In [ ]:
IMAGE_URI = DEPLOY_IMAGE


def upload_model(display_name, image_uri, model_uri):
    model = {
        "display_name": display_name,
        "metadata_schema_uri": "",
        "artifact_uri": model_uri,
        "container_spec": {
            "image_uri": image_uri,
            "command": [],
            "args": [],
            "env": [{"name": "env_name", "value": "env_value"}],
            "ports": [{"container_port": 8080}],
            "predict_route": "",
            "health_route": "",
        },
    }
    response = clients["model"].upload_model(parent=PARENT, model=model)
    print("Long running operation:", response.operation.name)
    upload_model_response = response.result(timeout=180)
    print("upload_model_response")
    print(" model:", upload_model_response.model)
    return upload_model_response.model


model_to_deploy_id = upload_model(
    "cifar10-" + TIMESTAMP, IMAGE_URI, model_path_to_deploy
)

获取`Model`资源信息

现在让我们仅获取您的模型的模型信息。使用以下参数的帮助函数`get_model`：

- `name`：`Model`资源的Vertex唯一标识符。

这个帮助函数调用了Vertex`Model`客户端服务的方法`get_model`，使用以下参数：

- `name`：`Model`资源的Vertex唯一标识符。

In [ ]:
def get_model(name):
    response = clients["model"].get_model(name=name)
    print(response)


get_model(model_to_deploy_id)

部署`Model`资源

现在部署训练后的Vertex自定义`Model`资源。 这需要两个步骤：

1. 为部署`Model`资源创建一个`Endpoint`资源。

2. 将`Model`资源部署到`Endpoint`资源。

### 创建 `Endpoint` 资源

使用这个辅助函数 `create_endpoint` 来创建一个端点资源，用于部署模型以提供预测，参数如下：

- `display_name`：`Endpoint` 资源的人类可读名称。

这个辅助函数使用端点客户端服务的 `create_endpoint` 方法，该方法接受以下参数：

- `display_name`：`Endpoint` 资源的人类可读名称。

创建 `Endpoint` 资源会返回一个长时间运行的操作，因为需要一些时间来为提供服务的 `Endpoint` 资源进行设置。您可以调用 `response.result()`，这是一个同步调用，当 Endpoint 资源准备就绪时会返回。该辅助函数返回 Vertex 完全合格的 `Endpoint` 资源标识符：`response.name`。

In [ ]:
ENDPOINT_NAME = "cifar10_endpoint-" + TIMESTAMP


def create_endpoint(display_name):
    endpoint = {"display_name": display_name}
    response = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)
    print("Long running operation:", response.operation.name)

    result = response.result(timeout=300)
    print("result")
    print(" name:", result.name)
    print(" display_name:", result.display_name)
    print(" description:", result.description)
    print(" labels:", result.labels)
    print(" create_time:", result.create_time)
    print(" update_time:", result.update_time)
    return result


result = create_endpoint(ENDPOINT_NAME)

现在获取您创建的“终端”资源的唯一标识符。

In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### 计算实例扩展

处理您的在线预测请求时，您有几种计算实例扩展的选择：

- 单个实例：在线预测请求在单个计算实例上处理。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为一。

- 手动扩展：在线预测请求在您手动指定的固定数量的计算实例上分拆。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为相同数量的节点。当模型首次部署到实例时，将提供固定数量的计算实例，并将在线预测请求均匀分配到它们。

- 自动扩展：在线预测请求在可扩展数量的计算实例上分拆。
  - 设置计算实例的最小（`MIN_NODES`）数量，在模型首次部署和取消部署时提供，并根据负载情况设置计算实例的最大（`MAX_NODES`）数量。

计算实例的最小数量对应于字段 `min_replica_count`，最大数量对应于字段 `max_replica_count`，在您的后续部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 部署`Model`资源到`Endpoint`资源

使用这个辅助函数`deploy_model`来将训练流水线中的`Model`资源部署到你为提供预测创建的`Endpoint`资源，使用以下参数：

- `model`: 要上传（部署）的模型的 Vertex 完全限定模型标识符。
- `deploy_model_display_name`: 部署模型的人类可读名称。
- `endpoint`: 要部署模型的 Vertex 完全限定终端标识符。

辅助函数调用`Endpoint`客户端服务的`deploy_model`方法，该方法接受以下参数：

- `endpoint`: 要部署`Model`资源的 Vertex 完全限定`Endpoint`资源标识符。
- `deployed_model`: 用于部署模型的需求规范。
- `traffic_split`: 流量分配至终端的百分比，表示为一个或多个键/值对的字典。
   - 如果只有一个模型，则指定为**{ "0": 100 }**，其中 "0" 指的是上传的这个模型，100 表示 100% 的流量。
   - 如果终端上存在其他模型，需要分配流量，则使用 `model_id` 指定为**{ "0": 百分比, model_id: 百分比, ... }**，其中 `model_id` 是要部署到的终端的现有模型的模型标识符。百分比必须加起来为 100。

现在我们深入研究`deployed_model`参数。此参数指定为具有最少必填字段的 Python 字典：

- `model`: 要部署的（上传）模型的 Vertex 完全限定模型标识符。
- `display_name`: 部署模型的人类可读名称。
- `disable_container_logging`: 禁用容器事件的日志记录，比如执行失败（默认情况下启用容器日志记录）。通常在调试部署时启用容器日志记录，然后在部署生产时禁用。
- `dedicated_resources`: 指的是为服务预测请求扩展的计算实例（副本）的数量。
  - `machine_spec`: 要配置的计算实例。使用先前设置的变量 `DEPLOY_GPU != None` 来使用 GPU；否则只分配一个 CPU。
  - `min_replica_count`: 最初要配置的计算实例的数量，在前面已设置为变量 `MIN_NODES`。
  - `max_replica_count`: 要扩展到的最大计算实例数量，在前面已设置为变量 `MAX_NODES`。

#### 流量分割

现在让我们更深入地了解`traffic_split`参数。此参数指定为一个 Python 字典。这可能一开始有点令人困惑。让我解释一下，您可以将模型的多个实例部署到一个终端，然后设置每个实例接收多少（百分比）流量。

为什么要这样做呢？也许您已经在生产环境中部署了之前的版本 —— 让我们称其为 v1。对于 v2，在评估模型表现时获得更好的结果，但在将其部署到生产中之前，您并不确定它是否真的更好。因此，在流量分配的情况下，您可能想将 v2 部署到与 v1 相同的终端，但它只获得 10% 的流量。这样，您可以在不干扰大部分用户的情况下监控其表现，直到做出最终决定。

#### 响应

该方法返回一个长时间运行的操作`response`。我们将通过调用`response.result()`来等待操作同步完成，这会阻塞直到模型部署完成。如果这是第一次将模型部署到终端，则可能需要额外几分钟来完成资源的配置。

In [ ]:
DEPLOYED_NAME = "cifar10_deployed-" + TIMESTAMP


def deploy_model(
    model, deployed_model_display_name, endpoint, traffic_split={"0": 100}
):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    deployed_model = {
        "model": model,
        "display_name": deployed_model_display_name,
        "dedicated_resources": {
            "min_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
            "machine_spec": machine_spec,
        },
        "disable_container_logging": False,
    }

    response = clients["endpoint"].deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split
    )

    print("Long running operation:", response.operation.name)
    result = response.result()
    print("result")
    deployed_model = result.deployed_model
    print(" deployed_model")
    print("  id:", deployed_model.id)
    print("  model:", deployed_model.model)
    print("  display_name:", deployed_model.display_name)
    print("  create_time:", deployed_model.create_time)

    return deployed_model.id


deployed_model_id = deploy_model(model_to_deploy_id, DEPLOYED_NAME, endpoint_id)

发出一个在线预测请求

现在对您部署的模型进行在线预测。

获取测试项目

您将使用数据集中测试（保留）部分的示例作为测试项目。

In [ ]:
test_image = x_test[0]
test_label = y_test[0]
print(test_image.shape)

In [ ]:
import base64

bytes = (test_image * 255).astype(np.uint8).tobytes()
b64str = base64.b64encode(np.ascontiguousarray(bytes)).decode("utf-8")

### 发送预测请求

好的，现在您有一个测试图片。使用这个辅助函数 `predict_image`，它接受以下参数：

- `image`：作为numpy数组的测试图片数据。
- `endpoint`：`Model` 资源部署到的 Vertex 完全限定标识符的 `Endpoint` 资源。
- `parameters_dict`：用于服务的附加参数。

这个函数调用预测客户端服务的 `predict` 方法，使用以下参数：

- `endpoint`：`Model` 资源部署到的 Vertex 完全限定标识符的 `Endpoint` 资源。
- `instances`：要预测的实例（编码图像）的列表。
- `parameters`：用于服务的附加参数。

要将图像数据传递给预测服务，在上一步中您将字节编码为base64 -- 这样当在网络上传输二进制数据时可以确保内容不被修改。您需要告诉服务二进制模型部署到哪里，内容已被base64编码，这样在服务二进制端会对其进行解码。

预测请求中的每个实例是以下形式的字典条目：

                        {serving_input: {'b64': content}}

- `input_name`：底层模型的输入层名称。
- `'b64'`：指示内容为base64编码的键。
- `content`：压缩的JPG图像字节作为base64编码字符串。

由于 `predict()` 服务可以接受多个图像（实例），您将会将您的单个图像作为一个包含一个图像的列表发送。作为最后一步，您需要将实例列表打包成谷歌的protobuf格式 -- 这是我们传递给 `predict()` 服务的内容。

`response` 对象将返回一个列表，列表中的每个元素对应请求中的相应图像。在每次预测的输出中，您将会看到：

- `predictions`：每个类别的预测置信度，介于0到1之间。

In [ ]:
def predict_image(image, endpoint, parameters_dict):
    # The format of each instance should conform to the deployed model's prediction input schema.
    instances_list = [{serving_input: {"b64": image}}]
    instances = [json_format.ParseDict(s, Value()) for s in instances_list]

    response = clients["prediction"].predict(
        endpoint=endpoint, instances=instances, parameters=parameters_dict
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    predictions = response.predictions
    print("predictions")
    for prediction in predictions:
        print(" prediction:", prediction)


predict_image(b64str, endpoint_id, None)

## 卸载 `Model` 资源

现在将您的 `Model` 资源从服务的 `Endpoint` 资源中卸载。使用这个帮助函数 `undeploy_model`，它接受以下参数：

- `deployed_model_id`：当 `Model` 资源部署到服务时由终端服务返回的模型部署标识符。
- `endpoint`：`Model` 部署到的 `Endpoint` 资源的 Vertex 完全限定标识符。

此函数调用终端客户服务的方法 `undeploy_model`，参数如下：

- `deployed_model_id`：当 `Model` 资源部署时终端服务返回的模型部署标识符。
- `endpoint`：`Model` 资源部署在其中的 `Endpoint` 资源的 Vertex 完全限定标识符。
- `traffic_split`：如何在 `Endpoint` 资源上分配流量给其余部署的模型。

由于这是 `Endpoint` 资源上唯一部署的模型，您可以通过将其设置为空来留空 `traffic_split`。

In [ ]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients["endpoint"].undeploy_model(
        endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={}
    )
    print(response)


undeploy_model(deployed_model_id, endpoint_id)

整理清理

要清理此项目中使用的所有GCP资源，您可以删除用于教程的[GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 端点
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME